In [8]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import sys

BASE_DIR = '../'

In [9]:
import os
sys.path.append('../src')
BASE_DIR = os.path.dirname(os.getcwd())
print(BASE_DIR)

print(os.getcwd())
from data import util
from data import analysis_util


/home/jenny/boston-crash-modeling
/home/jenny/boston-crash-modeling/notebooks


In [10]:
def print_summary(total_crashes, crashes, counts, inter_count, non_inter_count):
    """                                                                                                                                                                 
    Info about intersections vs non intersections and their crash rate                                                                                                  
    """
    
    print("Total number of crashes:\t\t\t\t" + str(total_crashes))
    print("Total number of crash locations:\t\t\t\t" + str(len(crashes)))
    print("Number of intersections:\t\t\t\t" + str(inter_count))
    print("Number of non-intersections:\t\t\t\t" + str(non_inter_count))
    print("Percentage of segments that are intersections:\t\t" \
        + str(round(100 * float(inter_count)/float(inter_count + non_inter_count))))
    print("")

    print("Number of intersections with at least 1 crash:\t\t" + str(counts['inter']))
    print('Number of intersections with more than 1 crash:\t\t' + str(counts['inter_plus']))
    print("Number of non-intersections with at least 1 crash:\t" + str(counts['non_inter']))
    print("Number of non-intersections with more than 1 crash:\t" + str(counts['non_inter_plus']))
    print("")

    total_percent = round(float(100 * (counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)))
    print("percent of all segments with crash:\t\t\t" + str(total_percent))

    # Percentage of intersections/non-intersections                                                                                                                     
    # that have at least one crash
    inter_percent = round(float(100 * counts['inter'])/float(inter_count))
    print("percent of intersections with crash:\t\t\t" + str(inter_percent))

    non_inter_percent = round(100 * float(counts['non_inter'])/float(non_inter_count))
    print("percentage of non-intersections with crash:\t\t" + str(non_inter_percent))

    print("percentage of intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['inter_plus'])/float(inter_count))))

    print("percentage of non-intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['non_inter_plus'])/float(non_inter_count))))


In [11]:
def print_concern_summary(results):
    """                                                                                                                                                                 
    What percentage of intersections with concerns had crashes                                                                                                          
    at varying counts of concerns?                                                                                                                                      
    """

    results = results[0:20]

    display(pd.DataFrame(results, columns=[
        '# of concerns at this segment',
        '% of segments w/ v0 complaint w/ crash',
        'total # of segments with this many or more concerns',
        '% of intersections with this many or more concerns with a crash',
        '# of intersections with this many concerns',
        '% of non-intersections with this many concerns with a crash',
        '# of non-intersections with this many concerns',
        '% of these locations that were at an intersection'
    ]))

In [12]:
def print_concerns_by_type(results):
    df = pd.DataFrame(results, columns=[
        'Request type',
        '% of segments with this type of request with a crash',
        '# of segments with this type of request with a crash',
        '# of segments with this type of a request total',
        '% of intersections with this type of request with a crash',
        '# of intersections with this type of request with a crash',
        '# of intersections with this type of request total',
        '% of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request total',
    ])
    df.sort_values(by="# of segments with this type of a request total", ascending=0)
    display(df)


In [13]:
def print_analysis_for_city(dir, crash_file, concern_file, category_field='REQUESTTYPE', years=None, yearfield='dateOccurred', cutoff=100):
    inters = util.read_segments(os.path.join(dir, 'processed/maps'), get_non_inter=False)
    non_inters = util.read_segments(os.path.join(dir, 'processed/maps'), get_inter=False)
    inter_count = len(inters[0])
    non_inter_count = len(non_inters[0])

    crash_file = os.path.join(dir, 'processed', crash_file)
    concern_file = os.path.join(dir, 'processed', concern_file)
    
    with open(crash_file, "r") as f:
        items = json.load(f)

    crash_data, crashes = util.group_json_by_location(
        items, yearfield=yearfield, years=years)

    with open(concern_file, "r") as f:
        items = json.load(f)
    concern_data, concerns = util.group_json_by_location(
        items,
        otherfields=[category_field])

    total_crashes, crash_locations, crash_results = analysis_util.summary_crash_rate(crashes)
   
    print_summary(total_crashes, crashes, crash_results, inter_count, non_inter_count)
    total_count, total_loc, inter_total, inter_loc, non_inter_total, non_inter_loc, concern_summary = analysis_util.summary_concern_counts(
        crashes, concerns)
    print("total concerns:" + str(len(concern_data)))
    print("concern locations:" + str(len(concerns)))
    results = analysis_util.concern_percentages(concern_summary)
    print_concern_summary(results)
 
    concerns_by_type = analysis_util.concern_counts_by_type(
        concern_data, crashes, category_field)
    concern_percent_by_type = analysis_util.concern_percentages_by_type(
        concerns_by_type, cutoff)
    print_concerns_by_type(concern_percent_by_type)


In [16]:
#print_analysis_for_city('../osm-data', 'crash_joined.json', 'concern_joined.json', years=[2016])
#print "-------------------------------"
print_analysis_for_city(
    '../data/boston', 
    'crash_joined.json',
    'concern_joined.json',
    category_field='category', years=[2016], cutoff=20)

Read in 7790 intersection, 0 non-intersection segments
Read in 0 intersection, 14676 non-intersection segments
Total number of crashes:				4349
Total number of crash locations:				2465
Number of intersections:				7790
Number of non-intersections:				14676
Percentage of segments that are intersections:		35

Number of intersections with at least 1 crash:		1327
Number of intersections with more than 1 crash:		533
Number of non-intersections with at least 1 crash:	1137
Number of non-intersections with more than 1 crash:	251

percent of all segments with crash:			11
percent of intersections with crash:			17
percentage of non-intersections with crash:		8
percentage of intersections with more than 1 crash:	7
percentage of non-intersections with more than 1 crash:	2
total concerns:14046
concern locations:6124


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,22,6124,32,2458,15,3666,40
1,2,29,2702,42,1149,20,1553,43
2,3,35,1465,50,651,23,814,44
3,4,41,919,57,422,27,497,46
4,5,47,612,64,291,32,321,48
5,6,50,416,67,196,35,220,47
6,7,55,306,72,147,39,159,48
7,8,59,234,77,111,44,123,47
8,9,63,178,79,84,50,94,47
9,10,67,140,83,65,53,75,46


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,Broken Sidewalk,35,18,51,38,5,13,34,13,38
1,Sidewalk Not Shoveled,26,16,61,36,8,22,21,8,39
2,Illegal Parking,17,28,164,25,13,52,13,15,112
3,Other,28,41,147,39,21,54,22,20,93
4,Pothole,33,104,318,47,50,107,26,54,211
5,Roadway Plowing/Salting,33,9,27,58,7,12,13,2,15
6,Traffic Signal,45,49,109,64,43,67,14,6,42
7,Abandoned Vehicle,32,9,28,29,2,7,33,7,21
8,Tree Pruning,25,5,20,38,3,8,17,2,12
9,Street Lights,25,15,59,26,6,23,25,9,36


In [17]:
print_analysis_for_city(
    '../data/cambridge', 
    'crash_joined.json',
    'concern_joined.json',
    category_field='category', years=[2016], cutoff=20)

Read in 1292 intersection, 0 non-intersection segments
Read in 0 intersection, 2409 non-intersection segments
Total number of crashes:				1588
Total number of crash locations:				532
Number of intersections:				1292
Number of non-intersections:				2409
Percentage of segments that are intersections:		35

Number of intersections with at least 1 crash:		140
Number of intersections with more than 1 crash:		36
Number of non-intersections with at least 1 crash:	391
Number of non-intersections with more than 1 crash:	126

percent of all segments with crash:			14
percent of intersections with crash:			11
percentage of non-intersections with crash:		16
percentage of intersections with more than 1 crash:	3
percentage of non-intersections with more than 1 crash:	5
total concerns:24596
concern locations:3124


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,16,3124,12,1137,19,1987,36
1,2,18,2710,12,971,21,1739,36
2,3,19,2314,13,809,23,1505,35
3,4,20,1979,13,665,24,1314,34
4,5,22,1697,14,553,26,1144,33
5,6,23,1447,14,448,27,999,31
6,7,24,1238,14,361,27,877,29
7,8,25,1058,14,303,29,755,29
8,9,26,908,15,239,30,669,26
9,10,28,760,14,190,32,570,25


,Request type,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total
0,Missed Compost Pickup (Green Bin),22,34,156,9,3,32,25,31,124
1,Icy or Unshoveled Sidewalk,22,232,1057,14,54,378,26,178,679
2,Abandoned Bicycle,25,60,238,18,13,73,28,47,165
3,Sidewalk Defect,24,230,967,16,46,292,27,184,675
4,Other,21,257,1221,11,49,435,26,208,786
5,Pothole,21,256,1238,14,64,472,25,192,766
6,Traffic Enforcement - Police (Internal),36,20,56,21,6,28,50,14,28
7,Traffic Sign Complaint,20,123,608,14,38,280,26,85,328
8,Missed Recycling Pickup,21,172,832,15,26,179,22,146,653
9,Tree Maintenance Issue,29,14,49,12,2,16,36,12,33
